In [332]:

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)


model_name = "EleutherAI/gpt-neo-1.3B"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [333]:
to_tokenize = " 28asdf"
input_ids = tokenizer.encode(to_tokenize, return_tensors="pt")
print(input_ids)

to_decode = [2579]
print(tokenizer.decode(to_decode, skip_special_tokens=True))

tensor([[2579,  292, 7568]])
 28


In [334]:
from transformers import (
    PreTrainedTokenizer,
    LogitsWarper,StoppingCriteria
)


# class NumberStoppingCriteria(StoppingCriteria):
#     """
#     This class can be used to stop generation when there is a repeated decimal point in the generated text.
#     """

#     def __init__(self, tokenizer: PreTrainedTokenizer):
#         self.tokenizer = tokenizer


#     def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
#         decoded = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
#         if ".." in decoded:
#             return True
        
#         if ' ' in decoded.strip():
#             return True
        
#         return False


class OutputNumbersTokens(LogitsWarper):
    def __init__(self, tokenizer: PreTrainedTokenizer, prompt: str):
        self.whitelist_tokens = [tokenizer.eos_token_id]
        self.tokenized_prompt = tokenizer(prompt, return_tensors="pt")

        for token_str, token_id in tokenizer.get_vocab().items():

            if (
                token_str.startswith("Ġ") and (
                    all(c.isdigit() or c == "." for c in token_str[1:]) and token_str.count(".") <= 1
                ) or (
                    all(c.isdigit() or c == "." for c in token_str) and token_str.count(".") <= 1
                )
            ):
                self.whitelist_tokens.append(token_id)

    def __call__(self, input_ids, scores):
        input_ids = input_ids[:, len(self.tokenized_prompt["input_ids"][0]):]
        scores[
            :, [i for i in range(len(scores[0])) if i not in self.whitelist_tokens]
        ] = -float("inf")
        return scores

a = OutputNumbersTokens(tokenizer, prompt)


In [326]:

a = OutputNumbersTokens(tokenizer, prompt)


found space28 2579


In [331]:
prompts = ["my grandma's age in years is", "today's temperature in degrees cels is", "today's temperature in f is", 
           "1 + 1 ="]

for prompt in prompts:
    number = model.generate(
        tokenizer.encode(prompt, return_tensors="pt"),
        max_new_tokens=5,
        num_return_sequences=1,
        logits_processor=[a],
        stopping_criteria=[NumberStoppingCriteria(tokenizer)],
    )   

    print(number)

    decoded_output = tokenizer.decode(number[0], skip_special_tokens=True)

    print(decoded_output.strip())


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 65 tensor(-3.7568)
 60 tensor(-3.8070)
 75 tensor(-3.9135)
 80 tensor(-4.0881)
 87 tensor(-4.1663)
 82 tensor(-4.2518)
 50 tensor(-4.2757)
 5 tensor(-4.3758)
 64 tensor(-4.3760)
 72 tensor(-4.4309)
tensor([[ 1820, 49890,   338,  2479,   287,   812,   318,  6135]])
my grandma's age in years is 65


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 0 tensor(-3.4328)
 5 tensor(-4.0209)
 36 tensor(-4.2786)
 19 tensor(-4.4433)
 59 tensor(-4.5570)
 24 tensor(-4.7431)
 33 tensor(-4.7621)
 56 tensor(-4.7745)
 47 tensor(-4.8474)
 53 tensor(-4.8719)
tensor([[40838,   338,  5951,   287,  7370,   269,  1424,   318,   657]])
today's temperature in degrees cels is 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 0 tensor(-5.0538)
 5 tensor(-6.4346)
 7 tensor(-7.2013)
 60 tensor(-7.4283)
 24 tensor(-7.6246)
 42 tensor(-7.6409)
 50 tensor(-7.6574)
 30 tensor(-7.6641)
 18 tensor(-7.7838)
 59 tensor(-7.8880)
tensor([[40838,   338,  5951,   287,   277,   318,   657]])
today's temperature in f is 0
 0 tensor(-1.6487)
 5 tensor(-3.0234)
 7 tensor(-4.6462)
 16 tensor(-5.9200)
 18 tensor(-6.3192)
 19 tensor(-6.6764)
 24 tensor(-6.7219)
 30 tensor(-7.0326)
2 tensor(-7.3029)
 50 tensor(-7.7194)
tensor([[  16, 1343,  352,  796,  657]])
1 + 1 = 0
